# Scraping Fandom for scripts

# Library

In [1]:
from bs4 import BeautifulSoup
import re
from urllib.request import urlopen
import pandas as pd

# Name of the all the episodes - transcript

In [3]:
# get the link
list_of_urls = []
name_url_dict = {}
# get the page through Beautiful soup
url = 'https://amphibia.fandom.com/wiki/Category:Transcripts'
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

prefix_url = 'https://amphibia.fandom.com/wiki/'

# use for loop to get the title for each of the page
for tag in soup.find_all("a"):
    title = tag.attrs.get("title") # the tag contain the word title
    if title is not None and title.endswith("/Transcript"):
#         print(title)
        episode_name = title.strip().replace("/Transcript", "").strip()  # get the name of the episode
#         print(f"{episode_name},")
        title = title.strip().replace(" ", "_")  # get the title for the url
        tmp_url = prefix_url+title
#         print(tmp_url)
        list_of_urls.append(tmp_url)
    
        name_url_dict[episode_name] = tmp_url

In [5]:
# replace ? with %3F
list_of_urls = [url.replace("?", "%3F") for url in list_of_urls]

In [6]:
# replace ? in the dict
for name, url in name_url_dict.items():
    if "?" in url:
        url = url.replace("?", "%3F")
        name_url_dict[name] = url

# Looping through a list

In [8]:
# Loop through the list of url to find those that have tables vs those that doesn't
no_table_url = []
with_table_url = []
for url in list_of_urls:
    try:
        page = urlopen(url)
        html = page.read().decode("utf-8")
        soup = BeautifulSoup(html, "html.parser")
        test_table = soup.find_all("table")
        df = pd.read_html(str(test_table))[2]
        
        # if the url does not have table
        if len(df) < 5 and (len(pd.read_html(str(test_table))[3]) < 5):
            no_table_url.append(url)
        elif len(df) < 5 and (len(pd.read_html(str(test_table))[3]) > 5):
            df = pd.read_html(str(test_table))[3]
        
        # if the url has table successfully
        if len(df) > 10:
            with_table_url.append(url)
        
        print(url, len(df))
    except Exception:
        no_table_url.append(url)
        print(url)
        pass

https://amphibia.fandom.com/wiki/A_Caravan_Named_Desire/Transcript 155
https://amphibia.fandom.com/wiki/A_Day_at_the_Aquarium/Transcript 2
https://amphibia.fandom.com/wiki/A_Night_at_the_Inn/Transcript 2
https://amphibia.fandom.com/wiki/Adventures_in_Catsitting/Transcript 189
https://amphibia.fandom.com/wiki/After_the_Rain/Transcript 2
https://amphibia.fandom.com/wiki/All_In/Transcript 142
https://amphibia.fandom.com/wiki/Amphibia_and_The_Owl_House_crossover_panel/Transcript 129
https://amphibia.fandom.com/wiki/Anne_Hunter/Transcript 197
https://amphibia.fandom.com/wiki/Anne_of_the_Year/Transcript 2
https://amphibia.fandom.com/wiki/Anne_or_Beast%3F/Transcript 179
https://amphibia.fandom.com/wiki/Anne_Theft_Auto/Transcript 2
https://amphibia.fandom.com/wiki/Anne_Vs._Wild/Transcript 33
https://amphibia.fandom.com/wiki/Anne-sterminator/Transcript 162
https://amphibia.fandom.com/wiki/Barrel's_Warhammer/Transcript 2
https://amphibia.fandom.com/wiki/Battle_of_the_Bands/Transcript 2
https://a

# Explore a page with no structure - still says who said what - in bold

In [14]:
url = 'https://amphibia.fandom.com/wiki/A_Night_at_the_Inn/Transcript'
# url = 'https://amphibia.fandom.com/wiki/Cane_Crazy/Transcript'
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

In [66]:
# function
def return_soup_given_url(url_input):
    """
    This function returns the soup given a url
    """
    page = urlopen(url_input)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    return soup
url = 'https://amphibia.fandom.com/wiki/Cane_Crazy/Transcript'
# return_soup_given_url(url)

In [16]:
# a function to return a df if a url has table
def return_df_with_table(input_soup):
    """
    This function returns a df with there is a "table" in the page
    """
    test_table = input_soup.find_all("table")
    df = pd.read_html(str(test_table))[2]
    return df
tmp_url = "https://amphibia.fandom.com/wiki/Hop_Luck/Transcript"
tmp_soup = return_soup_given_url(tmp_url)
return_df_with_table(tmp_soup)

,0,1
0,v • e — Episode transcripts —,v • e — Episode transcripts —
1,Season 1,1. Anne or Beast?/Best Fronds • 2. Cane Crazy/...
2,Season 2,1. Handy Anne/Fort in the Road • 2. The Ballad...
3,Season 3,1. The New Normal • 2. Hop 'Til You Drop/Turni...


In [17]:
# explore
# url = 'https://amphibia.fandom.com/wiki/Maddie_%26_Marcy/Transcript'
url = 'https://amphibia.fandom.com/wiki/The_Three_Armies/Transcript'
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")
test2 = soup.find_all('p')
# test2

for i,row in enumerate(test2):
    print(i, row.get_text())


https://amphibia.fandom.com/wiki/Family_Fishing_Trip/Transcript - this one does not have any speaker identifiable, the return_df_without_p function will return a short dataframe while return_df_with_p_and_b will return an empty dataframe

print([row.get_text() for i, row in enumerate(test3) if i==0]) when used with soup.find_all('p') will print everything for the pages without proper bold speaker, but will probably only print the first line when used for pages without proper bold speaker

In [18]:
# explore
url = 'https://amphibia.fandom.com/wiki/The_Three_Armies/Transcript'
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")
test3 = soup.find_all('b')

for i,row in enumerate(test3):
    print(f"{i}: {row.getText()} {row.nextSibling}")

0: This is a transcribed copy of The Three Armies.  —

1: Previous:  "
2: Next:  "
3: This section is currently incomplete.  —

4: ENOUGH!!! None
5: I SAID ENOUGH!!!  (uses the blue gem ability to push everyone away, everyone screaming in the process, to stop fighting)

6: All this time I thought I can solve your problems and heal this rift. 

7: But I was wrong. This is no quick fix. I can't solve this problem for you.  (deactivates her powers and falls down softly while Sasha catches her) Only you can. So when you're ready to work together and save the world, let me know.



In [67]:
url = 'https://amphibia.fandom.com/wiki/The_Three_Armies/Transcript'
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")
test2 = soup.find_all('p')
index_speaker_dict = {}
index_dialogue_dict = {}
for i,row in enumerate(test2):
#     print(i, row.get_text())
    string_conversation = row.get_text()
#     print(i, string_conversation)

    # if any of them is none then next
    if string_conversation is None or (":" not in string_conversation):
        continue
    clean_convo = string_conversation.strip()
    parts = clean_convo.split(':')
    speaker, dialogue = parts[0], parts[-1]

In [20]:
# MKII
def return_df_with_p_and_b(soup):
    """
    This function return a df for the soup that has "p" and "b" only
    """
    find_all_soup = soup.find_all('p')
    index_speaker_list = []
    index_dialogue_list = []
    
    for i,row in enumerate(find_all_soup):
        string_conversation = row.get_text()
#         print(i, string_conversation)
        
        # if any of them is none then next
        if string_conversation is None or (":" not in string_conversation):
            continue
        
        clean_convo = string_conversation.strip()
        parts = clean_convo.split(':')
        speaker, dialogue = parts[0], parts[-1]
            
        # if the speaker contains next/ previous or the speaker is long
        if 'Next' in speaker or 'Previous' in speaker or speaker.count(" ")>5:
            continue
        
#         print(i, speaker, dialogue, string_conversation)
        # put the speaker into list
        index_speaker_list.append(speaker)
        index_dialogue_list.append(dialogue)

    #turn them into dataframe
    tmp_df = pd.DataFrame([index_speaker_list, index_dialogue_list], index=['speaker', 'dialogue']).T
    
    return tmp_df
tmp_url = 'https://amphibia.fandom.com/wiki/The_Three_Armies/Transcript'
tmp_soup = return_soup_given_url(tmp_url)
return_df_with_p_and_b(tmp_soup)

,speaker,dialogue
0,Sasha,Listen up! Our spies tell us that Andrias pla...
1,[Grime],We're ready!
2,[Polly and Sprig],Oh yeah!
3,Anne,"Which means, it's time for us to join up with..."
4,Wally,Did you say newts... and toads... coming here...
...,...,...
126,Sasha,You heard the lady. Let's move out!
127,Everyone (except Anne and Sasha),Yes ma'am.
128,Sprig,Is now a good time to mention that we've also...
129,Anne,Let's give him a minute.


In [21]:
# tmp_soup = return_soup_given_url('https://amphibia.fandom.com/wiki/Cane_Crazy/Transcript')
tmp_soup = return_soup_given_url('https://amphibia.fandom.com/wiki/The_Three_Armies/Transcript')
find_all_soup = tmp_soup.find_all('b')
index_speaker_dict = {}
index_dialogue_dict = {}

for i,row in enumerate(find_all_soup):
    
    # handle speaker
    speaker = row.get_text().replace(":","").strip()
    if 'Next' in speaker or 'Previous' in speaker or len(speaker)>5:
        continue
    index_speaker_dict[i] = speaker
    
    # process the dialogue
    dialogue = row.next_sibling
    dialogue = dialogue.replace(":", "").strip()
    
    # put into dict
    index_dialogue_dict[i] = dialogue
    
#turn them into dataframe
tmp_df = pd.DataFrame({'speaker':pd.Series(index_speaker_dict),'dialogue':pd.Series(index_dialogue_dict)}).reset_index(drop=True)
tmp_df

/home/han/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


,speaker,dialogue


In [22]:
# test2 = soup.find_all('b')
# MKII
def return_df_without_p(soup):
    """ function
    This is for those that do not have a "p" but has "b"
    """
    find_all_soup = soup.find_all('b')
    index_speaker_list = []
    index_dialogue_list = []
    
    for i,row in enumerate(find_all_soup):
#         print(row.get_text(), row.next_sibling)
        
        # handle speaker
        speaker = row.get_text().replace(":","").strip()
        if 'Next' in speaker or 'Previous' in speaker or len(speaker)>5:
            continue
        index_speaker_dict[i] = speaker

        # process the dialogue
        dialogue = row.next_sibling
        dialogue = dialogue.replace(":", "").strip()

        # put into dict
        index_dialogue_dict[i] = dialogue


        # put the speaker into list
        index_speaker_list.append(speaker)
        index_dialogue_list.append(dialogue)
        
        # turn them into dataframe
    tmp_df = pd.DataFrame([index_speaker_list, index_dialogue_list], index=['speaker', 'dialogue']).T
        
    return tmp_df

tmp_url = 'https://amphibia.fandom.com/wiki/Cane_Crazy/Transcript'  # works
tmp_soup = return_soup_given_url(tmp_url)
return_df_without_p(tmp_soup)

,speaker,dialogue
0,Sprig,"Wow, Anne, look at all your cool stuff! What's..."
1,Anne,That's a toenail clipper.
2,Sprig,"Oh, okay, okay, sure. Oh! How about this?"
3,Anne,You know what? You can keep it.
4,Kids,Time to eat! Time to eat!
...,...,...
98,Sprig,Whoo! Yeah! I like that.
99,Polly,Yeah! I wasn't worried.
100,Anne,"Thanks, Hop Pop. So you're not mad about the c..."
101,Anne,"Ugh. Yes, sir!"


# Loop to sort through those with no tables

In [23]:
#Loop through the no table list
# no_bold_url_list = []
# bold_speaker_url_list = []
got_speaker_url_list = []
unknown_url_list = []
exception_url_list = []

for url in no_table_url:
    print(url)
    page = urlopen(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    
    try:
        # if the page got the p
        if (len(return_df_with_p_and_b(soup)) > 10):
            print("Got speaker - ", url, len(return_df_with_p_and_b(soup)))
            got_speaker_url_list.append(url)
            
        # if the page does not have p
        elif (len(return_df_with_p_and_b(soup)) < 10):
            if (len(return_df_without_p(soup)) > 10):
                print("Got speaker - ", url, len(return_df_without_p(soup)))
                got_speaker_url_list.append(url)
            elif (len(return_df_without_p(soup))< 10):
                print("unknown - ", url, len(return_df_without_p(soup)))
                unknown_url_list.append(url)
            
      
    except Exception:
        if (len(return_df_without_p(soup)) > 10):
                print("Got speaker - ", url, len(return_df_without_p(soup)))
                got_speaker_url_list.append(url)
        else:
            print("Exception - ", url)
            exception_url_list.append(url)
            pass


https://amphibia.fandom.com/wiki/A_Day_at_the_Aquarium/Transcript
unknown -  https://amphibia.fandom.com/wiki/A_Day_at_the_Aquarium/Transcript 0
https://amphibia.fandom.com/wiki/A_Night_at_the_Inn/Transcript
Got speaker -  https://amphibia.fandom.com/wiki/A_Night_at_the_Inn/Transcript 150
https://amphibia.fandom.com/wiki/After_the_Rain/Transcript
Got speaker -  https://amphibia.fandom.com/wiki/After_the_Rain/Transcript 143
https://amphibia.fandom.com/wiki/Anne_of_the_Year/Transcript
Got speaker -  https://amphibia.fandom.com/wiki/Anne_of_the_Year/Transcript 37
https://amphibia.fandom.com/wiki/Anne_Theft_Auto/Transcript
Got speaker -  https://amphibia.fandom.com/wiki/Anne_Theft_Auto/Transcript 118
https://amphibia.fandom.com/wiki/Barrel's_Warhammer/Transcript
unknown -  https://amphibia.fandom.com/wiki/Barrel's_Warhammer/Transcript 1
https://amphibia.fandom.com/wiki/Battle_of_the_Bands/Transcript
Got speaker -  https://amphibia.fandom.com/wiki/Battle_of_the_Bands/Transcript 122
https://

# Check the length of the tables from the ones that have structures

In [31]:
# go through the list with tables
short_table_url = []
long_table_url = []

# loop through all the tables to find length and append the short ones to a list to investigate
for url in with_table_url:
    tmp_soup = return_soup_given_url(url)
    tmp_df = return_df_with_table(tmp_soup)
    if len(tmp_df) > 2:
        print("table - ", len(tmp_df), " - ", url)
        long_table_url.append(url)
    else:
        short_table_url.append(url)
        
for url in short_table_url:
    tmp_soup = return_soup_given_url(url)
    tmp_df = return_df_with_table(tmp_soup)
    print("table - ", len(tmp_df), " - ", url)

table -  155  -  https://amphibia.fandom.com/wiki/A_Caravan_Named_Desire/Transcript
table -  189  -  https://amphibia.fandom.com/wiki/Adventures_in_Catsitting/Transcript
table -  142  -  https://amphibia.fandom.com/wiki/All_In/Transcript
table -  129  -  https://amphibia.fandom.com/wiki/Amphibia_and_The_Owl_House_crossover_panel/Transcript
table -  197  -  https://amphibia.fandom.com/wiki/Anne_Hunter/Transcript
table -  179  -  https://amphibia.fandom.com/wiki/Anne_or_Beast%3F/Transcript
table -  162  -  https://amphibia.fandom.com/wiki/Anne-sterminator/Transcript
table -  182  -  https://amphibia.fandom.com/wiki/Commander_Anne/Transcript
table -  178  -  https://amphibia.fandom.com/wiki/Contagi-Anne/Transcript
table -  132  -  https://amphibia.fandom.com/wiki/Escape_to_Amphibia/Transcript
table -  174  -  https://amphibia.fandom.com/wiki/Fight_at_the_Museum/Transcript
table -  181  -  https://amphibia.fandom.com/wiki/Fixing_Frobo/Transcript
table -  147  -  https://amphibia.fandom.com

In [32]:
# go through those without a table
for url in got_speaker_url_list:
    tmp_soup = return_soup_given_url(url)
    try:
        tmp_df = return_df_with_p_and_b(tmp_soup)
        if (len(tmp_df)) < 10:
            tmp_df = return_df_without_p(tmp_soup)
    except:
        tmp_df = return_df_without_p(tmp_soup)
    print("no table - ", len(tmp_df), " - ", url)

no table -  150  -  https://amphibia.fandom.com/wiki/A_Night_at_the_Inn/Transcript
no table -  143  -  https://amphibia.fandom.com/wiki/After_the_Rain/Transcript
no table -  37  -  https://amphibia.fandom.com/wiki/Anne_of_the_Year/Transcript
no table -  118  -  https://amphibia.fandom.com/wiki/Anne_Theft_Auto/Transcript
no table -  122  -  https://amphibia.fandom.com/wiki/Battle_of_the_Bands/Transcript
no table -  111  -  https://amphibia.fandom.com/wiki/Best_Fronds/Transcript
no table -  28  -  https://amphibia.fandom.com/wiki/Bizarre_Bazaar/Transcript
no table -  163  -  https://amphibia.fandom.com/wiki/Breakout_Star/Transcript
no table -  103  -  https://amphibia.fandom.com/wiki/Cane_Crazy/Transcript
no table -  166  -  https://amphibia.fandom.com/wiki/Civil_Wart/Transcript
no table -  144  -  https://amphibia.fandom.com/wiki/Combat_Camp/Transcript
no table -  155  -  https://amphibia.fandom.com/wiki/Cracking_Mrs._Croaker/Transcript
no table -  193  -  https://amphibia.fandom.com/wi

# Explore the short tables

In [33]:
short_table_url

['https://amphibia.fandom.com/wiki/Anne_Vs._Wild/Transcript',
 'https://amphibia.fandom.com/wiki/Cursed!/Transcript',
 'https://amphibia.fandom.com/wiki/Fight_or_Flight/Transcript',
 'https://amphibia.fandom.com/wiki/Girl_Time/Transcript',
 "https://amphibia.fandom.com/wiki/Grime's_Pupil/Transcript",
 'https://amphibia.fandom.com/wiki/Marcy_at_the_Gates/Transcript',
 'https://amphibia.fandom.com/wiki/Olm_Town_Road/Transcript',
 "https://amphibia.fandom.com/wiki/Plantar's_Last_Stand/Transcript",
 'https://amphibia.fandom.com/wiki/Snow_Day/Transcript',
 "https://amphibia.fandom.com/wiki/Sprig's_Birthday/Transcript",
 'https://amphibia.fandom.com/wiki/The_Dinner/Transcript',
 'https://amphibia.fandom.com/wiki/The_Second_Temple/Transcript',
 'https://amphibia.fandom.com/wiki/Wally_and_Anne/Transcript']

In [34]:
# url = "https://amphibia.fandom.com/wiki/Thai_Feud/Transcript"
# url = 'https://amphibia.fandom.com/wiki/Fight_or_Flight/Transcript'
url = 'https://amphibia.fandom.com/wiki/Froggy_Little_Christmas/Transcript'  # use the table 2,3,4
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")
test_table = soup.find_all("table")
df = pd.read_html(str(test_table))[2]
len(df)

67

In [35]:
for url in short_table_url:
    tmp_soup = return_soup_given_url(url)
    test_table = tmp_soup.find_all("table")
    tmp_df = pd.read_html(str(test_table))[3]
    print( len(tmp_df) , " - ", url)

33  -  https://amphibia.fandom.com/wiki/Anne_Vs._Wild/Transcript
104  -  https://amphibia.fandom.com/wiki/Cursed!/Transcript
43  -  https://amphibia.fandom.com/wiki/Fight_or_Flight/Transcript
74  -  https://amphibia.fandom.com/wiki/Girl_Time/Transcript
54  -  https://amphibia.fandom.com/wiki/Grime's_Pupil/Transcript
128  -  https://amphibia.fandom.com/wiki/Marcy_at_the_Gates/Transcript
70  -  https://amphibia.fandom.com/wiki/Olm_Town_Road/Transcript
86  -  https://amphibia.fandom.com/wiki/Plantar's_Last_Stand/Transcript
94  -  https://amphibia.fandom.com/wiki/Snow_Day/Transcript
167  -  https://amphibia.fandom.com/wiki/Sprig's_Birthday/Transcript
191  -  https://amphibia.fandom.com/wiki/The_Dinner/Transcript
82  -  https://amphibia.fandom.com/wiki/The_Second_Temple/Transcript
85  -  https://amphibia.fandom.com/wiki/Wally_and_Anne/Transcript


# Aggregate all the scrapable urls and place the dfs into a dict

In [36]:
# set up an empty dictionary for url and dfs
url_df_dict = {}

## Go through those with long table except for the Froggy Little Christmas

In [37]:
# for the long ones
for url in long_table_url:
    if 'Froggy_Little_Christmas' in url:
        continue
    tmp_soup = return_soup_given_url(url)
    tmp_df = return_df_with_table(tmp_soup)
    url_df_dict[url] = tmp_df

## Sort through the Froggy Litle Christmas

In [38]:
# for the episode of Froggy Little Christmas
url = 'https://amphibia.fandom.com/wiki/Froggy_Little_Christmas/Transcript'
froggy_df_list = []
tmp_soup = return_soup_given_url(url)
page_table = soup.find_all("table")

# it has multiple parts
for i in [2,3,4]:
    df = pd.read_html(str(page_table))[i]
    froggy_df_list.append(df)

# concat the df
final_df = pd.concat(froggy_df_list)

# append to the url_df_dict
url_df_dict[url] = final_df

In [39]:
len(url_df_dict['https://amphibia.fandom.com/wiki/Froggy_Little_Christmas/Transcript'])

278

## Go through the short ones

In [40]:
# for the long ones
for url in short_table_url:
    if 'Froggy_Little_Christmas' in url:
        continue
    tmp_soup = return_soup_given_url(url)
    test_table = tmp_soup.find_all("table")
    tmp_df = pd.read_html(str(test_table))[3]
    url_df_dict[url] = tmp_df

## Go through those with different structures

In [41]:
for url in got_speaker_url_list:
    tmp_soup = return_soup_given_url(url)
    try:
        tmp_df = return_df_with_p_and_b(tmp_soup)
        if (len(tmp_df)) < 10:
            tmp_df = return_df_without_p(tmp_soup)
        
    except:
        tmp_df = return_df_without_p(tmp_soup)
    finally:
        url_df_dict[url] = tmp_df

# Scrape the episode list

In [68]:
url = 'https://amphibia.fandom.com/wiki/List_of_Amphibia_episodes'  # 3, 5, 7
tmp_soup = return_soup_given_url(url)
test_table = tmp_soup.find_all("table")
tmp_df = pd.read_html(str(test_table))[7]

In [43]:
col_name = [x[1] for x in tmp_df.columns]

In [44]:
new_df = tmp_df.T.reset_index(drop=True).T.reset_index(drop=True)
new_df.columns = col_name
new_df

,Episode #,Segment,Disney,Prod.,Title,Synopsis,Airdate,Viewers (millions)
0,1,1,301,440A-301,"""The New Normal""",Anne and the Plantars get transported to Earth...,"October 2, 2021",0.43[66]
1,2,2a,302,440A-303[67],"""Hop 'Til You Drop""",Anne brings the Plantars to the mall for a cra...,"October 9, 2021",0.32[68]
2,2,2b,302,TBA,"""Turning Point""",Sasha and Grime escape from Andrias and hide o...,"October 9, 2021",0.32[68]
3,3,3a,303,TBA,"""Thai Feud""",Sprig tries to get rid of a food truck that's ...,"October 16, 2021",0.36[69]
4,3,3b,303,TBA,Adventures in Catsitting,The Plantars volunteer to take Domino to the v...,"October 16, 2021",0.36[69]
5,4,4a,304,TBA,"""Fight at the Museum""",Anne and the Plantars try to rob a museum.[65],"October 23, 2021",0.39[70]
6,4,4b,304,TBA,"""Temple Frogs""",The Boonchuys take the Plantars to the Thai Te...,"October 23, 2021",0.39[70]
7,5,5a,305,440A-309[67],"""Fixing Frobo""",Polly tries to bring Frobo back to life.[65],"October 30, 2021",0.33[71]
8,5,5b,305,TBA,"""Anne-sterminator""",A dangerous enemy finally locates Anne and the...,"October 30, 2021",0.33[71]
9,6,6a,306,TBA,"""Mr. X""","Anne takes the Plantars to the movies, while t...","November 6, 2021",0.39[73]


In [45]:
# loop through the seasons
url = 'https://amphibia.fandom.com/wiki/List_of_Amphibia_episodes'  # 3, 5, 7
tmp_soup = return_soup_given_url(url)
episodes_df_list = []
episode_page = tmp_soup.find_all("table")

for ind, num in enumerate([3,5,7]):
    tmp_df = pd.read_html(str(test_table))[num]
    col_name = [x[1] for x in tmp_df.columns]  # get the column names
    new_df = tmp_df.T.reset_index(drop=True).T.reset_index(drop=True)  # drop the hierarchical column names
    new_df.columns = col_name  # set the column names
    new_df["Season"] = ind+1
    episodes_df_list.append(new_df)

# concat all of them into one
episode_df = pd.concat(episodes_df_list)
episode_df

,Episode #,Segment,Disney,Prod.,Title,Synopsis,Airdate,Viewers (millions),Season
0,1,1a,103,TBA,"""Anne or Beast?""",Sprig tries to capture a mysterious beast to p...,"June 17, 2019[note 2]",0.39[7],1
1,1,1b,103,TBA,"""Best Fronds""",Sprig takes Anne swimming in a nearby lake to ...,"June 17, 2019[note 2]",0.39[7],1
2,2,2a,101,TBA,"""Cane Crazy""","When Anne breaks Hop Pop's favorite cane, she ...","June 18, 2019",0.46[8],1
3,2,2b,101,TBA,"""Flood, Sweat & Tears""","When Anne's bedroom floods, she and Sprig must...","June 18, 2019",0.46[8],1
4,3,3a,102,440A-105[9],"""Hop Luck""","To win a village cooking contest, Anne leads t...","June 19, 2019",0.37[10],1
...,...,...,...,...,...,...,...,...,...
26,15,15b,315,TBA,"""Fight or Flight""",Anne tries to rescue an old friend from one of...,"April 23, 2022[85]",0.30[89],3
27,16,16a,316,440A-331[90][67],"""The Three Armies""","Anne and Sasha must convince the frogs, newts ...","April 30, 2022[85]",0.25[91],3
28,16,16b,316,440A-332,"""The Beginning of the End""",The final battle begins.[83],"April 30, 2022[85]",0.25[91],3
29,17,17,317,440A-333[67]440A-334[92][67]440A-335[93],"""All In""[94][95]",Anne and her friends fight for their lives as ...,"May 7, 2022[85]",0.35[97],3


In [46]:
# replace some of the []
for col in ["Airdate", "Viewers (millions)", "Title", "Synopsis", "Prod."]:
    episode_df[col] = episode_df[col].str.replace(r"\[.*\]","")

/home/han/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [47]:
# remove ""
episode_df["Title"] = episode_df["Title"].str.replace(r"[\"]", '')

/home/han/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [48]:
episode_df

,Episode #,Segment,Disney,Prod.,Title,Synopsis,Airdate,Viewers (millions),Season
0,1,1a,103,TBA,Anne or Beast?,Sprig tries to capture a mysterious beast to p...,"June 17, 2019",0.39,1
1,1,1b,103,TBA,Best Fronds,Sprig takes Anne swimming in a nearby lake to ...,"June 17, 2019",0.39,1
2,2,2a,101,TBA,Cane Crazy,"When Anne breaks Hop Pop's favorite cane, she ...","June 18, 2019",0.46,1
3,2,2b,101,TBA,"Flood, Sweat & Tears","When Anne's bedroom floods, she and Sprig must...","June 18, 2019",0.46,1
4,3,3a,102,440A-105,Hop Luck,"To win a village cooking contest, Anne leads t...","June 19, 2019",0.37,1
...,...,...,...,...,...,...,...,...,...
26,15,15b,315,TBA,Fight or Flight,Anne tries to rescue an old friend from one of...,"April 23, 2022",0.30,3
27,16,16a,316,440A-331,The Three Armies,"Anne and Sasha must convince the frogs, newts ...","April 30, 2022",0.25,3
28,16,16b,316,440A-332,The Beginning of the End,The final battle begins.,"April 30, 2022",0.25,3
29,17,17,317,440A-333,All In,Anne and her friends fight for their lives as ...,"May 7, 2022",0.35,3


# Match url with name and episode and segment

In [49]:
name_url_df = pd.DataFrame(name_url_dict.items(), columns=["Title", "Episode_url"])
name_url_df

,Title,Episode_url
0,A Caravan Named Desire,https://amphibia.fandom.com/wiki/A_Caravan_Nam...
1,A Day at the Aquarium,https://amphibia.fandom.com/wiki/A_Day_at_the_...
2,A Night at the Inn,https://amphibia.fandom.com/wiki/A_Night_at_th...
3,Adventures in Catsitting,https://amphibia.fandom.com/wiki/Adventures_in...
4,After the Rain,https://amphibia.fandom.com/wiki/After_the_Rai...
...,...,...
104,Truck Stop Polly,https://amphibia.fandom.com/wiki/Truck_Stop_Po...
105,True Colors,https://amphibia.fandom.com/wiki/True_Colors/T...
106,Turning Point,https://amphibia.fandom.com/wiki/Turning_Point...
107,Wally and Anne,https://amphibia.fandom.com/wiki/Wally_and_Ann...


In [50]:
name_episode_url_df =  episode_df.merge(name_url_df, how='left', on='Title')
name_episode_url_df

,Episode #,Segment,Disney,Prod.,Title,Synopsis,Airdate,Viewers (millions),Season,Episode_url
0,1,1a,103,TBA,Anne or Beast?,Sprig tries to capture a mysterious beast to p...,"June 17, 2019",0.39,1,https://amphibia.fandom.com/wiki/Anne_or_Beast...
1,1,1b,103,TBA,Best Fronds,Sprig takes Anne swimming in a nearby lake to ...,"June 17, 2019",0.39,1,https://amphibia.fandom.com/wiki/Best_Fronds/T...
2,2,2a,101,TBA,Cane Crazy,"When Anne breaks Hop Pop's favorite cane, she ...","June 18, 2019",0.46,1,https://amphibia.fandom.com/wiki/Cane_Crazy/Tr...
3,2,2b,101,TBA,"Flood, Sweat & Tears","When Anne's bedroom floods, she and Sprig must...","June 18, 2019",0.46,1,"https://amphibia.fandom.com/wiki/Flood,_Sweat_..."
4,3,3a,102,440A-105,Hop Luck,"To win a village cooking contest, Anne leads t...","June 19, 2019",0.37,1,https://amphibia.fandom.com/wiki/Hop_Luck/Tran...
...,...,...,...,...,...,...,...,...,...,...
101,15,15b,315,TBA,Fight or Flight,Anne tries to rescue an old friend from one of...,"April 23, 2022",0.30,3,https://amphibia.fandom.com/wiki/Fight_or_Flig...
102,16,16a,316,440A-331,The Three Armies,"Anne and Sasha must convince the frogs, newts ...","April 30, 2022",0.25,3,https://amphibia.fandom.com/wiki/The_Three_Arm...
103,16,16b,316,440A-332,The Beginning of the End,The final battle begins.,"April 30, 2022",0.25,3,https://amphibia.fandom.com/wiki/The_Beginning...
104,17,17,317,440A-333,All In,Anne and her friends fight for their lives as ...,"May 7, 2022",0.35,3,https://amphibia.fandom.com/wiki/All_In/Transc...


In [51]:
# check for na/ null
name_episode_url_df[name_episode_url_df["Episode_url"].isna()]

,Episode #,Segment,Disney,Prod.,Title,Synopsis,Airdate,Viewers (millions),Season,Episode_url


In [52]:
name_episode_url_df.to_excel("name_episode_url.xlsx", index=False)

# Make the data df into one giant df

In [53]:
url_df_dict['https://amphibia.fandom.com/wiki/A_Caravan_Named_Desire/Transcript']

,Speaker,Dialogue
0,(The episode begins with Anne and the Plantars...,(The episode begins with Anne and the Plantars...
1,Hop Pop,"All right, kids! We're about to hit the Dry Sw..."
2,Anne,"That's okay, dude. It's still a swamp. (snaps)..."
3,(The next day...),(The next day...)
4,Anne,(dried out) (takes the canteen while Sprig sti...
...,...,...
150,Polly,Just like you.
151,"(Hop Pop grabs Sprig, Polly, and Anne and lead...","(Hop Pop grabs Sprig, Polly, and Anne and lead..."
152,Hop Pop,"Silly, kids, making jokes. Well, gotta go. Bye..."
153,(Anne and the Plantars leave in a hurry.),(Anne and the Plantars leave in a hurry.)


In [54]:
# put all of them into a list
transcript_df_list = []
for key, val in url_df_dict.items():
    val["row_num"] = val.index
    val["episode_url"] = key
    val.columns = ["Speaker", "Dialogue", "Dialogue_num_in_eps", "Episode_url"]
    transcript_df_list.append(val)

In [55]:
url_df_dict['https://amphibia.fandom.com/wiki/A_Caravan_Named_Desire/Transcript']

,Speaker,Dialogue,Dialogue_num_in_eps,Episode_url
0,(The episode begins with Anne and the Plantars...,(The episode begins with Anne and the Plantars...,0,https://amphibia.fandom.com/wiki/A_Caravan_Nam...
1,Hop Pop,"All right, kids! We're about to hit the Dry Sw...",1,https://amphibia.fandom.com/wiki/A_Caravan_Nam...
2,Anne,"That's okay, dude. It's still a swamp. (snaps)...",2,https://amphibia.fandom.com/wiki/A_Caravan_Nam...
3,(The next day...),(The next day...),3,https://amphibia.fandom.com/wiki/A_Caravan_Nam...
4,Anne,(dried out) (takes the canteen while Sprig sti...,4,https://amphibia.fandom.com/wiki/A_Caravan_Nam...
...,...,...,...,...
150,Polly,Just like you.,150,https://amphibia.fandom.com/wiki/A_Caravan_Nam...
151,"(Hop Pop grabs Sprig, Polly, and Anne and lead...","(Hop Pop grabs Sprig, Polly, and Anne and lead...",151,https://amphibia.fandom.com/wiki/A_Caravan_Nam...
152,Hop Pop,"Silly, kids, making jokes. Well, gotta go. Bye...",152,https://amphibia.fandom.com/wiki/A_Caravan_Nam...
153,(Anne and the Plantars leave in a hurry.),(Anne and the Plantars leave in a hurry.),153,https://amphibia.fandom.com/wiki/A_Caravan_Nam...


In [56]:
# get it into a huge df
dialogues_df = pd.concat(transcript_df_list).reset_index(drop=True)

In [57]:
dialogues_df

,Speaker,Dialogue,Dialogue_num_in_eps,Episode_url
0,(The episode begins with Anne and the Plantars...,(The episode begins with Anne and the Plantars...,0,https://amphibia.fandom.com/wiki/A_Caravan_Nam...
1,Hop Pop,"All right, kids! We're about to hit the Dry Sw...",1,https://amphibia.fandom.com/wiki/A_Caravan_Nam...
2,Anne,"That's okay, dude. It's still a swamp. (snaps)...",2,https://amphibia.fandom.com/wiki/A_Caravan_Nam...
3,(The next day...),(The next day...),3,https://amphibia.fandom.com/wiki/A_Caravan_Nam...
4,Anne,(dried out) (takes the canteen while Sprig sti...,4,https://amphibia.fandom.com/wiki/A_Caravan_Nam...
...,...,...,...,...
11786,The Curator,Looks like we just hit paydirt Frog Soos.,9,https://amphibia.fandom.com/wiki/Wax_Museum/Tr...
11787,Frog Soos,"Doo doo doo. Say Mr.Ponds, do you get the fee...",10,https://amphibia.fandom.com/wiki/Wax_Museum/Tr...
11788,The Curator,You've been licking yourself again Frog Soos?,11,https://amphibia.fandom.com/wiki/Wax_Museum/Tr...
11789,Frog Soos,(Chuckle) Caught me again Mr.Ponds.,12,https://amphibia.fandom.com/wiki/Wax_Museum/Tr...


In [58]:
# dialogues_df.to_excel("dialogues.xlsx", index=False)

In [59]:
total_df = dialogues_df.merge(name_episode_url_df, how="left", on="Episode_url")
total_df

,Speaker,Dialogue,Dialogue_num_in_eps,Episode_url,Episode #,Segment,Disney,Prod.,Title,Synopsis,Airdate,Viewers (millions),Season
0,(The episode begins with Anne and the Plantars...,(The episode begins with Anne and the Plantars...,0,https://amphibia.fandom.com/wiki/A_Caravan_Nam...,3,3b,203,440A-206,A Caravan Named Desire,Hop Pop signs the family up to join the theatr...,"July 25, 2020",0.44,2.0
1,Hop Pop,"All right, kids! We're about to hit the Dry Sw...",1,https://amphibia.fandom.com/wiki/A_Caravan_Nam...,3,3b,203,440A-206,A Caravan Named Desire,Hop Pop signs the family up to join the theatr...,"July 25, 2020",0.44,2.0
2,Anne,"That's okay, dude. It's still a swamp. (snaps)...",2,https://amphibia.fandom.com/wiki/A_Caravan_Nam...,3,3b,203,440A-206,A Caravan Named Desire,Hop Pop signs the family up to join the theatr...,"July 25, 2020",0.44,2.0
3,(The next day...),(The next day...),3,https://amphibia.fandom.com/wiki/A_Caravan_Nam...,3,3b,203,440A-206,A Caravan Named Desire,Hop Pop signs the family up to join the theatr...,"July 25, 2020",0.44,2.0
4,Anne,(dried out) (takes the canteen while Sprig sti...,4,https://amphibia.fandom.com/wiki/A_Caravan_Nam...,3,3b,203,440A-206,A Caravan Named Desire,Hop Pop signs the family up to join the theatr...,"July 25, 2020",0.44,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11786,The Curator,Looks like we just hit paydirt Frog Soos.,9,https://amphibia.fandom.com/wiki/Wax_Museum/Tr...,5,5b,205,TBA,Wax Museum,The family visits a mysterious roadside odditi...,"August 8, 2020",0.32,2.0
11787,Frog Soos,"Doo doo doo. Say Mr.Ponds, do you get the fee...",10,https://amphibia.fandom.com/wiki/Wax_Museum/Tr...,5,5b,205,TBA,Wax Museum,The family visits a mysterious roadside odditi...,"August 8, 2020",0.32,2.0
11788,The Curator,You've been licking yourself again Frog Soos?,11,https://amphibia.fandom.com/wiki/Wax_Museum/Tr...,5,5b,205,TBA,Wax Museum,The family visits a mysterious roadside odditi...,"August 8, 2020",0.32,2.0
11789,Frog Soos,(Chuckle) Caught me again Mr.Ponds.,12,https://amphibia.fandom.com/wiki/Wax_Museum/Tr...,5,5b,205,TBA,Wax Museum,The family visits a mysterious roadside odditi...,"August 8, 2020",0.32,2.0


In [60]:
# name_url_df_copy.to_excel("debug.xlsx", index = False)

In [61]:
pd.DataFrame(dialogues_df["Episode_url"].unique()).to_excel("debug2.xlsx", index=False)

In [62]:
total_df.columns

Index(['Speaker', 'Dialogue', 'Dialogue_num_in_eps', 'Episode_url',
       'Episode #', 'Segment', 'Disney', 'Prod.', 'Title', 'Synopsis',
       'Airdate', 'Viewers (millions)', 'Season'],
      dtype='object')

In [63]:
total_df["Title"].nunique()

87

In [64]:
cols_sort = ["Season", "Episode #", "Segment", "Dialogue_num_in_eps"]
total_df = total_df.sort_values(cols_sort)
total_df

,Speaker,Dialogue,Dialogue_num_in_eps,Episode_url,Episode #,Segment,Disney,Prod.,Title,Synopsis,Airdate,Viewers (millions),Season
812,Dragonflies buzz.,Dragonflies buzz.,0,https://amphibia.fandom.com/wiki/Anne_or_Beast...,1,1a,103,TBA,Anne or Beast?,Sprig tries to capture a mysterious beast to p...,"June 17, 2019",0.39,1.0
813,Cuts to Stumpy's.,Cuts to Stumpy's.,1,https://amphibia.fandom.com/wiki/Anne_or_Beast...,1,1a,103,TBA,Anne or Beast?,Sprig tries to capture a mysterious beast to p...,"June 17, 2019",0.39,1.0
814,Wally,"Goodnight, you frogs! See you in the morrow. (...",2,https://amphibia.fandom.com/wiki/Anne_or_Beast...,1,1a,103,TBA,Anne or Beast?,Sprig tries to capture a mysterious beast to p...,"June 17, 2019",0.39,1.0
815,Frogs,(laughing at Wally),3,https://amphibia.fandom.com/wiki/Anne_or_Beast...,1,1a,103,TBA,Anne or Beast?,Sprig tries to capture a mysterious beast to p...,"June 17, 2019",0.39,1.0
816,Cuts to Wally walking in the forest and playin...,Cuts to Wally walking in the forest and playin...,4,https://amphibia.fandom.com/wiki/Anne_or_Beast...,1,1a,103,TBA,Anne or Beast?,Sprig tries to capture a mysterious beast to p...,"June 17, 2019",0.39,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,Amity,"(sighs) No, I'm Amity Blight. I can do this. I...",124,https://amphibia.fandom.com/wiki/Amphibia_and_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
611,Tiny Nose,"Oh, there she is.",125,https://amphibia.fandom.com/wiki/Amphibia_and_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
612,Narration(Dana Terrace),Reveal: Luz and Eda walking back from their tr...,126,https://amphibia.fandom.com/wiki/Amphibia_and_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
613,Amity,I gotta go!,127,https://amphibia.fandom.com/wiki/Amphibia_and_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
# total_df.to_excel("total_df.xlsx", index=False)